In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer, one_hot
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout
import keras.optimizers
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
# Source file directory
path_train = "./training"
 
files_train = skds.load_files(path_train,load_content=False)
 
label_index = files_train.target
label_names = files_train.target_names
labelled_files = files_train.filenames

# data_tags = ["filename","language","wikitext"]
data_tags = ["language","wikitext"]
data_list = []

In [3]:
print(label_index)
print(label_names)
print(labelled_files)

[1 2 2 1 0 0 2 3 2 1 0 0 2 2 2 2 3 0 1 2 3 2 2 3 1 3 1 1 0 2 1 3 1 3 0 0 1
 1 3 3 3 2 0 0 3 1 0 1 2 1 1 0 2 3 0 0 3 3 3]
['english', 'french', 'german', 'romanian']
['./training\\french\\fr_clean_wrap_7.txt'
 './training\\german\\de_clean_wrap_15.txt'
 './training\\german\\de_clean_wrap_9.txt'
 './training\\french\\fr_clean_wrap_9.txt'
 './training\\english\\en_clean_wrap_7.txt'
 './training\\english\\en_clean_wrap_11.txt'
 './training\\german\\de_clean_wrap_14.txt'
 './training\\romanian\\ro_clean_wrap_11.txt'
 './training\\german\\de_clean_wrap_6.txt'
 './training\\french\\fr_clean_wrap_3.txt'
 './training\\english\\en_clean_wrap_13.txt'
 './training\\english\\en_clean_wrap_6.txt'
 './training\\german\\de_clean_wrap_10.txt'
 './training\\german\\de_clean_wrap_7.txt'
 './training\\german\\de_clean_wrap_13.txt'
 './training\\german\\de_clean_wrap_8.txt'
 './training\\romanian\\ro_clean_wrap_13.txt'
 './training\\english\\en_clean_wrap_3.txt'
 './training\\french\\fr_clean_wrap_1.txt'
 

In [4]:
# Read and add data from file to a list
i=0
for f in labelled_files:
    # data_list.append((f,label_names[label_index[i]],Path(f).read_text(encoding="utf8")))
    data_list.append((label_names[label_index[i]],Path(f).read_text(encoding="utf8")))
    i += 1
 
# We have training data available as dictionary filename, category, data
data = pd.DataFrame.from_records(data_list, columns=data_tags)

In [6]:
data_english = data.loc[data['language'] == "english"].reset_index(drop=True)
data_german = data.loc[data['language'] == "german"].reset_index(drop=True)
data_french = data.loc[data['language'] == "french"].reset_index(drop=True)
data_romanian = data.loc[data['language'] == "romanian"].reset_index(drop=True)

data_sorted=pd.concat([data_english, data_german, data_french, data_romanian]).sort_index(kind='mergesort').reset_index(drop=True)
data_sorted.head(20)

,language,wikitext
0,english,Zealand and Australia for the first time The b...
1,german,die westliche Linke könne die Sowjetunion kein...
2,french,parlement de Bosnie-Herzégovine Les électeurs ...
3,romanian,în secolul al XIV-lea și comiți ai comitatului...
4,english,became part of the Héritage des Celtes led by ...
5,german,Deklination in der Regel nicht genau an den So...
6,french,de pays Lors du décès en de Marie de Nemours l...
7,romanian,suflet curat un revolver încărcat sau o grenad...
8,english,to build a stone house on his Mānoa land in Ca...
9,german,Metallbindung würde man zunächst anderes erwar...


In [8]:
train_size = int(len(data) * .8)
 
train_posts = data_sorted['wikitext'][:train_size]
train_tags = data_sorted['language'][:train_size]

 
test_posts = data_sorted['wikitext'][train_size:]
test_tags = data_sorted['language'][train_size:]


print(test_tags)

47    romanian
48     english
49      german
50      french
51    romanian
52     english
53      german
54      french
55    romanian
56      german
57      french
58    romanian
Name: language, dtype: object


In [312]:
# 4 languages
num_labels = 4
vocab_size = 385456
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=None, lower=True, split=' ')
#tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_posts)

In [313]:
#x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
#x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

#x_train = tokenizer.texts_to_sequences(train_posts)
#x_test = tokenizer.texts_to_sequences(test_posts)

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [314]:
x_train.shape

(47, 385456)

In [315]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [328]:
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=385456))
model.add(Dense(units=4, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 0s - loss: 1.4439 - accuracy: 0.4043
Epoch 2/100
 - 0s - loss: 1.0220 - accuracy: 0.9149
Epoch 3/100
 - 0s - loss: 0.3526 - accuracy: 1.0000
Epoch 4/100
 - 0s - loss: 0.3518 - accuracy: 1.0000
Epoch 5/100
 - 0s - loss: 0.3512 - accuracy: 1.0000
Epoch 6/100
 - 0s - loss: 0.3503 - accuracy: 1.0000
Epoch 7/100
 - 0s - loss: 0.3497 - accuracy: 1.0000
Epoch 8/100
 - 0s - loss: 0.3490 - accuracy: 1.0000
Epoch 9/100
 - 0s - loss: 0.3483 - accuracy: 1.0000
Epoch 10/100
 - 0s - loss: 0.3475 - accuracy: 1.0000
Epoch 11/100
 - 0s - loss: 0.3468 - accuracy: 1.0000
Epoch 12/100
 - 0s - loss: 0.3462 - accuracy: 1.0000
Epoch 13/100
 - 0s - loss: 0.3454 - accuracy: 1.0000
Epoch 14/100
 - 0s - loss: 0.3447 - accuracy: 1.0000
Epoch 15/100
 - 0s - loss: 0.3439 - accuracy: 1.0000
Epoch 16/100
 - 0s - loss: 0.3433 - accuracy: 1.0000
Epoch 17/100
 - 0s - loss: 0.3426 - accuracy: 1.0000
Epoch 18/100
 - 0s - loss: 0.3419 - accuracy: 1.0000
Epoch 19/100
 - 0s - loss: 0.3411 - accuracy: 1.0000
Ep

In [301]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_162 (Dense)            (None, 512)               197353984 
_________________________________________________________________
activation_14 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_77 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_163 (Dense)            (None, 512)               262656    
_________________________________________________________________
activation_15 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_78 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_164 (Dense)            (None, 4)               

In [321]:
model = Sequential()
model.add(Dense(500,input_dim=385456,kernel_initializer="glorot_uniform",activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(300,kernel_initializer="glorot_uniform",activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(100,kernel_initializer="glorot_uniform",activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(4,kernel_initializer="glorot_uniform",activation="softmax"))
model_optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=model_optimizer,
              metrics=['accuracy'])
# let's fit the data
# history variable will help us to plot results later
history = model.fit(x_train,y_train,
          epochs=12,
          validation_split=0.10,
          batch_size=64,
          verbose=2,
          shuffle=True)

Train on 42 samples, validate on 5 samples
Epoch 1/12
 - 4s - loss: 1.7879 - accuracy: 0.2619 - val_loss: 1.0792 - val_accuracy: 0.4000
Epoch 2/12
 - 3s - loss: 1.1214 - accuracy: 0.5952 - val_loss: 0.9680 - val_accuracy: 0.8000
Epoch 3/12
 - 2s - loss: 1.2764 - accuracy: 0.4048 - val_loss: 0.8934 - val_accuracy: 1.0000
Epoch 4/12
 - 2s - loss: 1.0836 - accuracy: 0.6190 - val_loss: 0.8361 - val_accuracy: 1.0000
Epoch 5/12
 - 2s - loss: 1.0661 - accuracy: 0.5714 - val_loss: 0.7840 - val_accuracy: 1.0000
Epoch 6/12
 - 3s - loss: 1.0707 - accuracy: 0.4524 - val_loss: 0.7354 - val_accuracy: 1.0000
Epoch 7/12
 - 2s - loss: 1.0277 - accuracy: 0.5476 - val_loss: 0.6895 - val_accuracy: 1.0000
Epoch 8/12
 - 2s - loss: 1.0072 - accuracy: 0.5238 - val_loss: 0.6410 - val_accuracy: 1.0000
Epoch 9/12
 - 2s - loss: 1.0001 - accuracy: 0.6190 - val_loss: 0.5922 - val_accuracy: 1.0000
Epoch 10/12
 - 2s - loss: 0.8490 - accuracy: 0.7381 - val_loss: 0.5443 - val_accuracy: 1.0000
Epoch 11/12
 - 2s - loss: 

In [329]:
#### now we will face the TRUTH. What is our model real accuracy tested on unseen data?
scores = model.evaluate(x_test, y_test, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

prediction = model.predict(tokenizer.texts_to_matrix("Ich möchte ein Bier"))

predicted_label = text_labels[np.argmax(prediction[0])]
print("Predicted label: " + predicted_label)


12/12 [==============================] - 0s 20ms/step
accuracy: 100.00%
Predicted label: german


In [ ]:
# show plot accuracy changes during training
plt.plot(history.history['accuracy'],'g')
plt.plot(history.history['val_accuracy'],'r')
plt.title('accuracy across epochs')
plt.ylabel('accuracy level')
plt.xlabel('# epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

In [ ]:
# show plot of loss changes during training
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [305]:
prediction = model.predict(tokenizer.texts_to_matrix("Habe nun, ach! Philosophie, Juristerei und Medizin, Und leider auch"))

predicted_label = text_labels[np.argmax(prediction[0])]
print("Predicted label: " + predicted_label)

Predicted label: english
